# (Optional) Load Data from Kaggle into Google Colab

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'aiim-emotion-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F77823%2F8553100%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240623%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240623T094401Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D805efb2257fe18bee5d9dd463e0a463db317c19cd0d9327850d64b4b620849cc1bfebdda40e3d3cbe566046d15f5424d8ec02f36785658c0a555ef4141ff3ec45b8e54ddf7af6db109b13fc132583255e056c00dafcb30082cc93f4f666991e66b63308438447692255ab0c12b1fb9a533fd5897498e85e162a4210da0999009e91831b641d41528b6d311bad0f5ab3cba4e848e29eb862e4cdfda98cbb0390921adf5a1079314c4187f661bd62364cc3758683df954292d965a0bd79036f2c7e29d80c49a8812f9026c4663e10c374fc27d0d7b38b231dfc8110d29e08682d31c964258b81d56add6dd51faf9d39cd1ddcffe0ef844bed684e9aa09e9f75086'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
    os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
    pass
try:
    os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
    pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
                with ZipFile(tfile) as zfile:
                    zfile.extractall(destination_path)
            else:
                with tarfile.open(tfile.name) as tarfile:
                    tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Visualizing some images

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import image_dataset_from_directory


base_directory = '/kaggle/input/aiim-emotion-classification/aiim-emotion-classification/'


# Laden der Datensätze ohne direkten Split
dataset = image_dataset_from_directory(
    directory=base_directory + 'train/',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=None,  # Keine Batch-Größe, um alle Daten zu laden
    shuffle=True,
    seed=420
)

def count_images_per_class(dataset, class_names):
    class_counts = {class_name: 0 for class_name in class_names}
    for image, label in dataset:
        class_idx = tf.argmax(label).numpy()
        class_name = class_names[class_idx]
        class_counts[class_name] += 1
    return class_counts

# Namen der Klassen
class_names = dataset.class_names
print("Klassen:", class_names)

class_counts = count_images_per_class(dataset, class_names)
print("Anzahl der Bilder im Datensatz pro Klasse:", class_counts)

def plot_images(images, titles, rows=1, cols=5, figsize=(15, 5)):
    plt.figure(figsize=figsize)
    for i in range(len(images)):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(images[i].numpy().squeeze(), cmap='gray')
        plt.title(titles[i])
        plt.axis('off')
    plt.show()
    
image_batch, label_batch = next(iter(dataset.batch(5)))

plot_images(image_batch, titles=[class_names[label.numpy().argmax()] for label in label_batch])